<a href="https://colab.research.google.com/github/bjelikj/DT2119/blob/master/Labs/Lab3/Lab3_V2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [11]:
import os
from google.colab import drive

ROOT = '/content/drive'
PROJ = 'My Drive/DT2119_Lab3'
PROJECT_PATH = os.path.join(ROOT, PROJ)
!mkdir '{PROJECT_PATH}'
drive.mount(ROOT)

GIT_REPOSITORY = "DT2119/Labs/Lab3"
GIT_PATH = "https://github.com/bjelikj/DT2119.git"
GIT_SUBFOLDER = 'DT2119/Labs/Lab3'

print('Directory where project will be stored: ', PROJECT_PATH)

print("Creating temp directory and cloning...")
%cd '{PROJECT_PATH}'
!mkdir ./temp
%cd 'temp'
%pwd
!git clone "{GIT_PATH}"
!ls
print("Moving files to Drive...")
%cd '{PROJECT_PATH}'
!mv ./temp/* "{PROJECT_PATH}"
print("Removing temp directory...")
%cd '{PROJECT_PATH}'
!rm -r ./temp
# !rsync -aP --exclude=data/ "{PROJECT_PATH}"/*  ./
!rsync -aP "{PROJECT_PATH}"/*  ./

mkdir: cannot create directory ‘/content/drive/My Drive/DT2119_Lab3’: File exists
Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Directory where project will be stored:  /content/drive/My Drive/DT2119_Lab3
Creating temp directory and cloning...
/content/drive/My Drive/DT2119_Lab3
/content/drive/My Drive/DT2119_Lab3/temp
Cloning into 'DT2119'...
remote: Enumerating objects: 128, done.
remote: Counting objects: 100% (128/128), done.
remote: Compressing objects: 100% (93/93), done.
remote: Total 216 (delta 64), reused 82 (delta 30), pack-reused 88
Receiving objects: 100% (216/216), 53.24 MiB | 14.80 MiB/s, done.
Resolving deltas: 100% (106/106), done.
DT2119
Moving files to Drive...
/content/drive/My Drive/DT2119_Lab3
Removing temp directory...
/content/drive/My Drive/DT2119_Lab3
sending incremental file list


In [12]:
WORKING_SUBFOLDER = os.path.join(PROJECT_PATH, GIT_SUBFOLDER)
print(WORKING_SUBFOLDER)
%cd '{WORKING_SUBFOLDER}'
!git pull

/content/drive/My Drive/DT2119_Lab3/DT2119/Labs/Lab3
/content/drive/My Drive/DT2119_Lab3/DT2119/Labs/Lab3
Already up to date.


In [0]:
"""
Necessary files:
- lab3_proto.py
- lab3_tools.py
- lab1_proto.py
- lab1_tools.py
- prondict.py

If not uploaded to Google Drive:
- lab2_models_all.npz
- lab3_example.npz
"""
"""
if IN_COLAB:
    !rm lab1_proto.py
    !rm lab1_tools.py
    !rm lab2_proto.py
    !rm lab2_tools.py
    !rm lab3_proto.py
    !rm lab3_tools.py
    !rm lab3_example.npz
    !rm lab2_models_all.npz
    !ls
"""
from google.colab import files
files.upload()

Saving lab1_proto.py to lab1_proto.py
Saving lab1_tools.py to lab1_tools.py
Saving lab2_proto.py to lab2_proto.py
Saving lab2_tools.py to lab2_tools.py
Saving lab3_proto.py to lab3_proto.py
Saving lab3_tools.py to lab3_tools.py
Saving prondict.py to prondict.py


{'lab1_proto.py': b'# DT2119, Lab 1 Feature Extraction\r\nimport numpy as np\r\nimport matplotlib.pyplot as plt\r\nimport scipy.signal as ssi\r\nfrom scipy import fftpack\r\nfrom lab1_tools import trfbank\r\nfrom scipy.fftpack.realtransforms import dct\r\nfrom lab1_tools import *\r\nfrom scipy.spatial.distance import euclidean\r\nfrom scipy.spatial.distance import cdist\r\n\r\n# Function given by the exercise ----------------------------------\r\n\r\ndef mspec(samples, winlen=400, winshift=200, preempcoeff=0.97, nfft=512, samplingrate=20000):\r\n    """Computes Mel Filterbank features.\r\n\r\n    Args:\r\n        samples: array of speech samples with shape (N,)\r\n        winlen: lenght of the analysis window\r\n        winshift: number of samples to shift the analysis window at every time step\r\n        preempcoeff: pre-emphasis coefficient\r\n        nfft: length of the Fast Fourier Transform (power of 2, >= winlen)\r\n        samplingrate: sampling rate of the original signal\r\n\r

In [0]:
from lab1_proto import *
from lab2_proto import *
from lab3_proto import *
from lab1_tools import *
from lab2_tools import *
from lab3_tools import *
from prondict import prondict

import numpy as np
import tensorflow as tf
import keras

## Check examples


In [0]:
example = np.load('/content/drive/My Drive/Lab3/lab3_example.npz', allow_pickle=True)['example'].item()

In [0]:
phoneHMMs = np.load('/content/drive/My Drive/Lab3/lab2_models_all.npz', allow_pickle=True)['phoneHMMs'].item()
phones = sorted(phoneHMMs.keys())
nstates = {phone: phoneHMMs[phone]['means'].shape[0] for phone in phones}
stateList = [ph + '_' + str(id) for ph in phones for id in range(nstates[ph])]
print(stateList)

['ah_0', 'ah_1', 'ah_2', 'ao_0', 'ao_1', 'ao_2', 'ay_0', 'ay_1', 'ay_2', 'eh_0', 'eh_1', 'eh_2', 'ey_0', 'ey_1', 'ey_2', 'f_0', 'f_1', 'f_2', 'ih_0', 'ih_1', 'ih_2', 'iy_0', 'iy_1', 'iy_2', 'k_0', 'k_1', 'k_2', 'n_0', 'n_1', 'n_2', 'ow_0', 'ow_1', 'ow_2', 'r_0', 'r_1', 'r_2', 's_0', 's_1', 's_2', 'sil_0', 'sil_1', 'sil_2', 'sp_0', 't_0', 't_1', 't_2', 'th_0', 'th_1', 'th_2', 'uw_0', 'uw_1', 'uw_2', 'v_0', 'v_1', 'v_2', 'w_0', 'w_1', 'w_2', 'z_0', 'z_1', 'z_2']


In [0]:
filename = '/content/drive/My Drive/tidigits/disc_4.1.1/tidigits/train/man/nw/z43a.wav'
samples, samplingrate = loadAudio(filename)
lmfcc = mfcc(samples)
print("LMFCC Shape: ", lmfcc.shape)

wordTrans = list(path2info(filename)[2])
print("Sequence of digits: ", wordTrans)

print("Pronunciation dictionary: ", prondict)

phoneTrans = words2phones(wordTrans, prondict)
print("Phone level transcription: ", phoneTrans)
utteranceHMM = concatHMMs(phoneHMMs, phoneTrans)
stateTrans = [phone + '_' + str(stateid) for phone in phoneTrans
              for stateid in range(nstates[phone])]

LMFCC Shape:  (178, 13)
Sequence of digits:  ['z', '4', '3']
Pronunciation dictionary:  {'o': ['ow'], 'z': ['z', 'iy', 'r', 'ow'], '1': ['w', 'ah', 'n'], '2': ['t', 'uw'], '3': ['th', 'r', 'iy'], '4': ['f', 'ao', 'r'], '5': ['f', 'ay', 'v'], '6': ['s', 'ih', 'k', 's'], '7': ['s', 'eh', 'v', 'ah', 'n'], '8': ['ey', 't'], '9': ['n', 'ay', 'n']}
Phone level transcription:  ['sil', 'z', 'iy', 'r', 'ow', 'f', 'ao', 'r', 'th', 'r', 'iy', 'sil', 'sp']


In [0]:
# Compare with examples
obsloglik = log_multivariate_normal_density_diag(lmfcc, utteranceHMM['means'], utteranceHMM['covars'])
viterbi_loglik, viterbi_path = viterbi(obsloglik, np.log(utteranceHMM['startprob']), np.log(utteranceHMM['transmat']))
#viterbiStateTrans = list(np.array(stateTrans)[viterbi_path])

print('Comparing obsloglik...', compare(obsloglik, example['obsloglik'][:, :-2]))
print('Comparing viterbiPath...', compare(viterbi_path, example['viterbiPath']))
#print('Comparing viterbiStateTrans...', np.all(viterbiStateTrans == example['viterbiStateTrans']))

frames = frames2trans(viterbiStateTrans, outfilename= ''.join(path2info(filename)[2:]) + '.lab')

Comparing obsloglik... False
Comparing viterbiPath... False


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:3: RuntimeWarning: divide by zero encountered in log
  This is separate from the ipykernel package so we can avoid doing imports until


NameError: ignored

In [0]:
obsloglik

array([[-82.85615934, -77.77520492, -78.78912809, ..., -77.77520492,
        -78.78912809, -77.77520492],
       [-82.64629092, -75.68935466, -78.4682569 , ..., -75.68935466,
        -78.4682569 , -75.68935466],
       [-81.28144137, -74.7891467 , -76.77156599, ..., -74.7891467 ,
        -76.77156599, -74.7891467 ],
       ...,
       [-78.94201025, -88.97211724, -80.73016898, ..., -88.97211724,
        -80.73016898, -88.97211724],
       [-77.97751766, -86.49199766, -79.0998656 , ..., -86.49199766,
        -79.0998656 , -86.49199766],
       [-76.34772424, -81.64468937, -77.4851362 , ..., -81.64468937,
        -77.4851362 , -81.64468937]])

In [0]:
example['obsloglik']

array([[-82.85615934, -77.77520492, -78.78912809, ..., -82.85615934,
        -77.77520492, -78.78912809],
       [-82.64629092, -75.68935466, -78.4682569 , ..., -82.64629092,
        -75.68935466, -78.4682569 ],
       [-81.28144137, -74.7891467 , -76.77156599, ..., -81.28144137,
        -74.7891467 , -76.77156599],
       ...,
       [-78.94201025, -88.97211724, -80.73016898, ..., -78.94201025,
        -88.97211724, -80.73016898],
       [-77.97751766, -86.49199766, -79.0998656 , ..., -77.97751766,
        -86.49199766, -79.0998656 ],
       [-76.34772424, -81.64468937, -77.4851362 , ..., -76.34772424,
        -81.64468937, -77.4851362 ]])

In [0]:
stateTrans[10]

In [0]:
frames2trans(viterbiStateTrans, outfilename='z43a.lab')

# 4.3 Feature Extraction

In [0]:
import os
traindata = []
for root, dirs, files in os.walk('tidigits/disc_4.1.1/tidigits/train'):
  for file in files:
    if file.endswith('.wav'):
      filename = os.path.join(root, file)
      samples, samplingrate = loadAudio(filename)
#...your code for feature extraction and forced alignment
      traindata.append({'filename': filename, 'lmfcc': lmfcc,
                        'mspec': 'mspec', 'targets': targets})
      
    

# 4.4 Training and Validation Sets

In [0]:
# Split the training data into a training set (roughly 90%) and validation set (remaining 10%).
# Make sure that there is a similar distribution of men and women in both sets, and that each
# speaker is only included in one of the two sets. The last requirement is to ensure that we do not
# get artificially good results on the validation set. Explain how you selected the two data sets.

# 4.5 Acoustic Context (Dynamic Features)

In [0]:
# For each utterance and time step, stack 7 MFCC or filterbank features symmetrically distributed
# around the current time step. That is, at time n, stack the features at times [n􀀀3; n􀀀
# 2; n􀀀1; n; n+1; n+2; n+3]). At the beginning and end of each utterance, use mirrored feature
# vectors in place of the missing vectors. For example at the beginning use feature vectors with
# indexes [3; 2; 1; 0; 1; 2; 3] for the first time step, [2; 1; 0; 1; 2; 3; 4] for the second time step, and so
# on. The “boundary effect” is usually not very important because each utterance begins and ends
# with silence.

# 4.6 Feature Standardisation